# IE 7300 - Assignment 10

In [2]:
from datetime import date
studentName="shucheng zhang"
studentEmail="zhang.shuche@northeastern.edu"
homework=10
contributedPercentage=100
print("IE 7300: Statistical learning in engineering")
print("\n")
print(f'Homework - {homework}')
print(f'Student name :{studentName}')
print(f'Student Email :{studentEmail}')
print("\n"*15)
print(f'Percentage of Effort Contributed by Student : {contributedPercentage}%')    
print(f'Submission Date: {date.today()}')

IE 7300: Statistical learning in engineering


Homework - 10
Student name :shucheng zhang
Student Email :zhang.shuche@northeastern.edu
















Percentage of Effort Contributed by Student : 100%
Submission Date: 2023-12-04


## Import Modules

In [3]:
# Importing the necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import time
import itertools

#Preprocessing methods
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
from typing import Dict, Any, List, Tuple
from sklearn.base import clone
import copy
from scipy import stats
from abc import ABC,abstractmethod
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler

## Import Dataset

In [4]:
# Read the letter recognition csv file
column_names = ["letter", "x-box", "y-box", "width", "high", 
                "onpix", "x-bar", "y-bar", "x2bar", "y2bar", 
                "xybar", "x2ybr", "xy2br", "x-ege", "xegvy",
                "y-ege", "yegvx"]
letter = pd.read_csv("letter-recognition.csv", names = column_names)
letter.head(10)

,letter,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
5,S,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7
6,B,4,2,5,4,4,8,7,6,6,7,6,6,2,8,7,10
7,A,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
8,J,2,2,4,4,2,10,6,2,6,12,4,8,1,6,1,7
9,M,11,15,13,9,7,13,2,6,2,12,1,9,8,1,1,8


## Data Preprocessing

In [6]:
# Checking null values.
letter.isnull().sum()

letter    0
x-box     0
y-box     0
width     0
high      0
onpix     0
x-bar     0
y-bar     0
x2bar     0
y2bar     0
xybar     0
x2ybr     0
xy2br     0
x-ege     0
xegvy     0
y-ege     0
yegvx     0
dtype: int64

No missing values, qualified dataset no extra action needed

## NN Implementation

In [7]:
# Implement the Nerual net class
class NeuralNet():
    '''
    A multiclass neural network with custom SGD and metrics
    '''
    # Initialize default parameters
    def __init__(self, layers=[13, 8, 3], learning_rate=0.001, iterations=100, activation='relu'):
        self.params = {}
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.loss = []
        self.layers = layers
        self.activation_func = activation
    
    # Initialize weights
    def init_weights(self):
        np.random.seed(1)
        self.params["W1"] = np.random.randn(self.layers[0], self.layers[1]) 
        self.params['b1'] = np.random.randn(self.layers[1],)
        self.params['W2'] = np.random.randn(self.layers[1], self.layers[2]) 
        self.params['b2'] = np.random.randn(self.layers[2],)
    
    # Define relu activation function
    def relu(self, Z):
        return np.maximum(0, Z)

    # relu derivatives
    def dRelu(self, Z):
        Z[Z <= 0] = 0
        Z[Z > 0] = 1
        return Z

    # Define softmax activation function in the out layer
    def softmax(self, Z):
        expZ = np.exp(Z - np.max(Z))
        return expZ / expZ.sum(axis=1, keepdims=True)
    
    # Define tanh activation function
    def tanh(self, Z):
        return np.tanh(Z)

    # tanh derivatives
    def dTanh(self, Z):
        return 1 - np.tanh(Z) ** 2
    
    # Define sigmoid activation function
    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))
    
    # sigmoid derivatives
    def dSigmoid(self, Z):
        sig = self.sigmoid(Z)
        return sig * (1 - sig)

    # Initialize forward propagation
    def forward_propagation(self):
        Z1 = self.X.dot(self.params['W1']) + self.params['b1']

        if self.activation_func == 'relu':
            A1 = self.relu(Z1)
        elif self.activation_func == 'tanh':
            A1 = self.tanh(Z1)
        elif self.activation_func == 'sigmoid':
            A1 = self.sigmoid(Z1)
        elif self.activation_func == 'softmax': 
            A1 = self.softmax(Z1)
        else:
            raise ValueError("Unsupported activation function")

        Z2 = A1.dot(self.params['W2']) + self.params['b2']
        yhat = self.softmax(Z2)
        loss = self.entropy_loss(self.y, yhat)
        self.params['Z1'], self.params['A1'], self.params['Z2'] = Z1, A1, Z2
        return yhat, loss

    # Entropy loss function calculation
    def entropy_loss(self, y, yhat):
        nsample = len(y)
        yhat_clipped = np.clip(yhat, 1e-8, 1 - 1e-8)
        loss = -1 / nsample * (y * np.log(yhat_clipped) + (1 - y) * np.log(1 - yhat_clipped)).sum()
        return loss

    # Initialize backward propagation function
    def back_propagation(self, yhat):
        # Compute the derivative of the loss with respect to the output of the network
        dl_wrt_z2 = yhat - self.y

        # Compute gradients for the second layer
        dl_wrt_A1 = dl_wrt_z2.dot(self.params['W2'].T)
        dl_wrt_w2 = self.params['A1'].T.dot(dl_wrt_z2)
        dl_wrt_b2 = np.sum(dl_wrt_z2, axis=0)

        # Compute gradients for the first layer
        if self.activation_func == 'relu':
            dl_wrt_z1 = dl_wrt_A1 * self.dRelu(self.params['Z1'])
        elif self.activation_func == 'tanh':
            dl_wrt_z1 = dl_wrt_A1 * self.dTanh(self.params['Z1'])
        elif self.activation_func == 'sigmoid':
            dl_wrt_z1 = dl_wrt_A1 * self.dSigmoid(self.params['Z1'])
        else:
            raise ValueError("Unsupported activation function")

        dl_wrt_w1 = self.X.T.dot(dl_wrt_z1)
        dl_wrt_b1 = np.sum(dl_wrt_z1, axis=0)

        # Update the weights and biases
        self.params['W1'] -= self.learning_rate * dl_wrt_w1
        self.params['W2'] -= self.learning_rate * dl_wrt_w2
        self.params['b1'] -= self.learning_rate * dl_wrt_b1
        self.params['b2'] -= self.learning_rate * dl_wrt_b2

    # Initialize the fit function
    def fit(self, X, y, optimizer='sgd'):
        self.X = X
        self.y = y
        self.init_weights()

        if optimizer == 'sgd':
            for i in range(self.iterations):
                for j in range(X.shape[0]):
                    self.X = X[j:j+1]
                    self.y = y[j:j+1]
                    yhat, loss = self.forward_propagation()
                    self.back_propagation(yhat)
                    self.loss.append(loss)
        elif optimizer == 'gd':
            for i in range(self.iterations):
                yhat, loss = self.forward_propagation()
                self.back_propagation(yhat)
                self.loss.append(loss)
        else:
            raise ValueError("Unsupported optimizer")

    # Initialize the predict function
    def predict(self, X):
        Z1 = X.dot(self.params['W1']) + self.params['b1']
        A1 = self.relu(Z1) if self.activation_func == 'relu' else self.tanh(Z1)
        Z2 = A1.dot(self.params['W2']) + self.params['b2']
        pred = self.softmax(Z2)
        return np.round(pred)

    # Accuracy calculation function
    def accuracy(self, y, yhat):
        acc = np.sum(y == yhat) / len(y)
        return acc

    # Confusion matrix generation function
    def compute_confusion_matrix(self, true, predictions):
        K = len(np.unique(true))  # Number of classes 
        result = np.zeros((K, K))
        for i in range(len(true)):
            result[true[i]][predictions[i]] += 1
        return result

    # Classification report generation function
    def compute_classification_report(self, true, predictions):
        matrix = self.compute_confusion_matrix(true, predictions)
        num_classes = matrix.shape[0]
        precision = np.zeros(num_classes)
        recall = np.zeros(num_classes)
        f1_score = np.zeros(num_classes)

        for i in range(num_classes):
            if sum(matrix[:, i]) == 0:
                precision[i] = np.nan  # Avoid division by zero
            else:
                precision[i] = matrix[i, i] / sum(matrix[:, i])

            if sum(matrix[i, :]) == 0:
                recall[i] = np.nan  # Avoid division by zero
            else:
                recall[i] = matrix[i, i] / sum(matrix[i, :])

            if (precision[i] + recall[i]) == 0:
                f1_score[i] = np.nan  # Avoid division by zero
            else:
                f1_score[i] = 2 * precision[i] * recall[i] / (precision[i] + recall[i])

        # Calculate average values, ignoring NaNs
        avg_precision = np.nanmean(precision)
        avg_recall = np.nanmean(recall)
        avg_f1_score = np.nanmean(f1_score)

        report = {"average_precision": avg_precision, "average_recall": avg_recall, "average_f1-score": avg_f1_score}
        return report



## Data train test split

In [8]:
# Define Features and Lables
X = letter.drop(columns=["letter"]) 
y = letter["letter"]


# Scaling dataset
sc = StandardScaler()
X = sc.fit_transform(X)



# Splitting the dataset again
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert Pandas Series to NumPy array and then reshape
y_train_array = y_train.to_numpy().reshape(-1, 1)
y_test_array = y_test.to_numpy().reshape(-1, 1)

# One-hot encode the labels
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train_array)
y_test_encoded = encoder.transform(y_test_array)

r:\Anaconda\envs\myenv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Fit and Predict

In [10]:
# Define train fit and predict function
def train_and_evaluate_model(X_train, y_train_encoded, X_test, y_test_encoded, layers, activation, optimizer):
    model = NeuralNet(layers=layers, activation=activation)
    model.fit(X_train, y_train_encoded, optimizer=optimizer)  # Use the modified fit method
    predictions = model.predict(X_test)
    accuracy = model.accuracy(y_test_encoded.argmax(axis=1), predictions.argmax(axis=1))
    confusion_matrix = model.compute_confusion_matrix(y_test_encoded.argmax(axis=1), predictions.argmax(axis=1))
    classification_report = model.compute_classification_report(y_test_encoded.argmax(axis=1), predictions.argmax(axis=1))
    print(f"Model with {layers} layers, {activation} activation, {optimizer} optimizer")
    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", confusion_matrix)
    print("Classification Report:\n", classification_report)

# Updated configurations
configurations = [
    ([16, 5, 26], 'relu', 'sgd'),
    ([16, 10, 26], 'relu', 'sgd'),
    ([16, 15, 26], 'relu', 'sgd'),
    ([16, 25, 26], 'relu', 'sgd'),
    ([16, 5, 26], 'tanh', 'sgd'),
    ([16, 10, 26], 'tanh', 'sgd'),
    ([16, 15, 26], 'tanh', 'sgd'),
    ([16, 25, 26], 'tanh', 'sgd'),
    ([16, 5, 26], 'sigmoid', 'sgd'),
    ([16, 15, 26], 'sigmoid', 'sgd'),
    ([16, 25, 26], 'sigmoid', 'sgd'),
    ([16, 5, 26], 'relu', 'gd'),
    ([16, 10, 26], 'relu', 'gd'),
    ([16, 15, 26], 'relu', 'gd'),
    ([16, 5, 26], 'tanh', 'gd'),
    ([16, 10, 26], 'tanh', 'gd'),
    ([16, 15, 26], 'tanh', 'gd'),
    ([16, 5, 26], 'sigmoid', 'gd'),
    ([16, 10, 26], 'sigmoid', 'gd'),
    ([16, 15, 26], 'sigmoid', 'gd')
]


# Train and evaluate each configuration
for layers, activation, optimizer in configurations:
    train_and_evaluate_model(X_train, y_train_encoded, X_test, y_test_encoded, layers, activation, optimizer)




Model with [16, 5, 26] layers, relu activation, sgd optimizer
Accuracy: 0.52075
Confusion Matrix:
 [[148.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 69.  80.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   1.   0.   2.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 37.   0.  79.   0.   2.   2.   0.   0.   0.   0.  11.   0.   0.   0.
    0.   0.   0.   0.   0.   2.   2.   1.   1.   0.   0.   0.]
 [ 50.   1.   0.  98.   0.   0.   0.   0.   0.   0.   0.   0.   2.   0.
    0.   5.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 59.   0.   0.   0.  79.   0.   1.   0.   0.   0.   1.   0.   0.   0.
    0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 53.   1.   0.   5.   0.  69.   0.   0.   0.   1.   0.   0.   0.   0.
    0.   5.   0.   1.   1.   4.   0.   0.   0.   0.   0.   0.]
 [116.   0.   4.   0.   5.   0.  32.   0.   0.   0.   2.   0.   0.   0.
    0.   0.   0.   

C:\Users\2000z\AppData\Local\Temp\ipykernel_5956\2652997046.py:33: RuntimeWarning: invalid value encountered in divide
  return expZ / expZ.sum(axis=1, keepdims=True)


Model with [16, 5, 26] layers, relu activation, gd optimizer
Accuracy: 0.03725
Confusion Matrix:
 [[149.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [153.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [137.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [156.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [141.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [140.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [160.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0

## Model performance metrics outcome analysis

### **Model with [16, 5, 26] layers, relu activation, sgd optimizer**

* `Accuracy:` 0.52075
* `Average Precision:` 0.8197
* `Average Recall:` 0.5213
* `Average F1-score:` 0.5760
* `observation:` Moderate accuracy with balanced precision and recall. This configuration has room for improvement.

### **Model with [16, 10, 26] layers, relu activation, sgd optimizer**

* `Accuracy:` 0.69825
* `Average Precision:` 0.8524
* `Average Recall:` 0.6977
* `Average F1-score:` 0.7459
* `Observation:` A significant improvement in all metrics compared to the first model. Increasing the middle layer's size seems beneficial.

### **Model with [16, 15, 26] layers, relu activation, sgd optimizer**

* `Accuracy:` 0.77775
* `Average Precision:` 0.8672
* `Average Recall:` 0.7768
* `Average F1-score:` 0.8048
* `Observation:` Further improvement in performance, indicating that a larger middle layer is more effective for this specific task.

### **Model with [16, 25, 26] layers, relu activation, sgd optimizer**

* `Accuracy:` 0.83825
* `Average Precision:` 0.8942
* `Average Recall:` 0.8371
* `Average F1-score:` 0.8553
* `Observation:` This configuration achieves the best performance among relu activation models. There's a consistent increase in performance metrics with the increase in the size of the middle layer.

### **Model with [16, 5, 26] layers, tanh activation, sgd optimizer**

* `Accuracy:` 0.389
* `Average Precision:` 0.7432
* `Average Recall:` 0.3857
* `Average F1-score:` 0.4874
* `Observation:` A significant drop in performance compared to relu models, indicating that tanh activation might not be as effective for this letter dataset.

### **Model with [16, 10, 26] layers, tanh activation, sgd optimizer**

* `Accuracy:` 0.6765
* `Average Precision:` 0.8448
* `Average Recall:` 0.6754
* `Average F1-score:` 0.7257
* `Observation:` Improvement over the previous tanh model, but still not as effective as relu models.

### **Model with [16, 15, 26] layers, tanh activation, sgd optimizer**

* `Accuracy:` 0.743
* `Average Precision:` 0.8740
* `Average Recall:` 0.7421
* `Average F1-score:` 0.7855
* `Observation:` The best performing tanh model, but still falls short compared to the best relu models.

### **Model with [16, 25, 26] layers, tanh activation, sgd optimizer**

* `Accuracy:` 0.79975
* `Average Precision:` 0.8769
* `Average Recall:` 0.7988
* `Average F1-score:` 0.8240
* `Observation:` Closest in performance to relu models but relu still outperforms in overall metrics.

### **Model with [16, 5, 26] layers, sigmoid activation, sgd optimizer**

* `Accuracy:` 0.40225
* `Average Precision:` 0.5771
* `Average Recall:` 0.3978
* `Average F1-score:` 0.4103
* `Observation:` Low performance, indicating that sigmoid activation is not suitable for this letter dataset as well.

### **Model with [16, 15, 26] layers, sigmoid activation, sgd optimizer**

* `Accuracy:` 0.65725
* `Average Precision:` 0.7721
* `Average Recall:` 0.6587
* `Average F1-score:` 0.6558
* `Observation:` Moderate improvement but still significantly lower than relu and tanh models.

### **Model with [16, 25, 26] layers, sigmoid activation, sgd optimizer**

* `Accuracy:` 0.67975
* `Average Precision:` 0.7721
* `Average Recall:` 0.6798
* `Average F1-score:` 0.6780
* `Observation:` Slight improvement over the 15-neuron sigmoid model but still not competitive with relu or tanh models.

</br>

### **Model with [16, 5, 26] layers, relu activation, gd optimizer**

* `Accuracy:` 0.03725
* `Average Precision:` 0.03725
* `Average Recall:` 0.03846
* `Average F1-score:` 0.07182
* `Observation:` Extremely poor performance, with accuracy and other metrics being significantly low. This suggests that the combination of relu activation and GD is ineffective for this letter dataset.

### **Model with [16, 10, 26] layers, relu activation, gd optimizer**

* `Accuracy:` 0.03725
* `Average Precision:` 0.03725
* `Average Recall:` 0.03846
* `Average F1-score:` 0.07182
* `Observation:` Similar to the previous model, this configuration also shows very poor performance.

### **Model with [16, 15, 26] layers, relu activation, gd optimizer**

* `Accuracy:` 0.03725
* `Average Precision:` 0.03725
* `Average Recall:` 0.03846
* `Average F1-score:` 0.07182
* `Observation:` Consistent with the previous relu models, this configuration also fails to produce satisfactory results.

### **Model with [16, 5, 26] layers, tanh activation, gd optimizer**

* `Accuracy:` 0.2555
* `Average Precision:` 0.5797
* `Average Recall:` 0.2543
* `Average F1-score:` 0.4307
* `Observation:` Shows a moderate increase in performance compared to the relu models, indicating that tanh might be a more suitable activation function when used with GD.

### **Model with [16, 10, 26] layers, tanh activation, gd optimizer**

* `Accuracy:` 0.58225
* `Average Precision:` 0.7383
* `Average Recall:` 0.5760
* `Average F1-score:` 0.6697
* `Observation:` There's a significant improvement in performance with this configuration. This suggests that increasing in the middle layer, combined with tanh activation and GD, can yield better results.

### **Model with [16, 15, 26] layers, tanh activation, gd optimizer**

* `Accuracy:` 0.653
* `Average Precision:` 0.8065
* `Average Recall:` 0.6509
* `Average F1-score:` 0.6643
* `Observation:` Demonstrates the best performance among the tanh models, indicating a positive correlation between the number of neurons in the middle layer and the model's effectiveness when using tanh and GD.

### **Model with [16, 5, 26] layers, sigmoid activation, gd optimizer**

* `Accuracy:` 0.37775
* `Average Precision:` 0.4812
* `Average Recall:` 0.3730
* `Average F1-score:` 0.4012
* `Observation:` Shows better performance than relu models but is still considerably underperforming compared to tanh models, indicating that sigmoid is not the most effective choice for this dataset with GD.

### **Model with [16, 10, 26] layers, sigmoid activation, gd optimizer**

* `Accuracy:` 0.545
* `Average Precision:` 0.6426
* `Average Recall:` 0.5427
* `Average F1-score:` 0.5274
* `Observation:` An improvement in performance is observed with an increased number of neurons, but it still dropped behind the tanh models.

### **Model with [16, 15, 26] layers, sigmoid activation, gd optimizer**

* `Accuracy:` 0.5875
* `Average Precision:` 0.7404
* `Average Recall:` 0.5860
* `Average F1-score:`  0.5860
* `Observation:` Shows the best performance among the sigmoid models, yet it still does not match the effectiveness of the tanh models.

## Analysis and Discussion

### 1. Evaluation and Comparison of Model Performance

`Hidden Nodes:`

* Increase in Hidden Nodes: According to my project dataset: Letter recognition, there's a clear trend that increasing the number of hidden nodes generally improves all performance metrics (accuracy, precision, recall, F1-score) across all activation functions and optimization functions. This is likely due to the model's increased capacity to capture complex patterns in the data.

`Activation Functions:`

* ReLU (Rectified Linear Unit): Based on the outcome, this specific model with ReLU consistently outperform those with tanh and sigmoid activation functions. ReLU is known for its efficiency in training deep neural networks due to its simplicity and ability to mitigate the vanishing gradient problem.
* Tanh (Hyperbolic Tangent): According to the model output, the Tanh models perform better than sigmoid but are less effective than ReLU. Tanh is symmetrical around zero and can handle negative values more effectively than sigmoid, but it seems to fall short of the performance achieved by ReLU in this letter dataset.
* Sigmoid: Sigmoid models showed the weakest performance. This might be due to the vanishing gradient problem, as sigmoid activation functions saturate at both ends (0 and 1), leading to small gradients and slow learning.

`Optimization Functions:`

* Different Types of Gradient Descent: The performance of the models also varied with different types of gradient descent. The differences in performance can be attributed to how each method approaches the optimization process.
* Gradient Descent: For GD, it updates the model parameters using the entire dataset at once. While it provides stable error gradients and a steady convergence, it can be computationally intensive and slow, especially with large datasets.
* Stochastic Gradient Descent (SGD): For SGD, it updates parameters for each training example. It's faster and can escape local minima better than gradient descent, but it can be more efficient in its path towards convergence.

### 2. Interpretation and Discussion of Findings

`Impact of Model Complexity:`

* The increase in model complexity (more hidden nodes) generally leads to better performance. However, it's also important to balance complexity with the risk of overfitting, where the model performs well on training data but poorly on unseen data.

`Activation Function Suitability:`

* ReLU's superior performance suggests it's more suitable for this specific letter recognition dataset. This might be due to its linear, non-saturating form, which helps in faster convergence during training.
Tanh and sigmoid's lower performance indicates potential issues with vanishing gradients, especially for sigmoid.

`Optimization Method Efficacy:`

* The choice of gradient descent method significantly impacts model performance. Gradient descent might be too slow and computationally intensive for large datasets, while stochastic gradient descent, though faster, might lead to inconsistent convergence. However, in this case, SGD did a decent job in this model.

`General Trends:`

* Precision, recall, and F1-scores follow similar trends to accuracy. High precision suggests a low rate of false positives, while high recall indicates a low rate of false negatives. The F1-score provides a balance between precision and recall, which is crucial for imbalanced datasets.

`Dataset-Specific Considerations:`

* The nature of this dataset also played a significant role in how well each model configuration performs. Certain datasets might inherently favor certain activation functions due to its characteristics, complexities, etc.


To conclude, to gain a more comprehensive understanding, experimenting with different optimizers, learning rates, and even more diverse architectures would be beneficial. The findings emphasize the importance of model architecture and parameters' choices tailored to the specific outcome of the dataset. ReLU emerges as the most effective activation function in this experiments, with an increase in hidden nodes correlating with improved performance across all metrics.